In [1]:
import numpy as np

In [2]:
def prep_lines(lines):
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines


def fortran_to_python(ODEs, spec):
    print(spec)

    # print('type1',type(ODEs[spec]))
    if isinstance(ODEs[spec],str):
        ## NOT fractional abundances, but in units of cm^-3
        ODEs[spec] = ODEs[spec].replace('*HNR', '')
        ## split string
        ODEs[spec] = ODEs[spec].split(' ')

    # print('type2',type(ODEs[spec]))
    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## remove other fortran commands
    while 'END' in ODEs[spec]:
        ODEs[spec].remove('END')
    while 'RETURN' in ODEs[spec]:
        ODEs[spec].remove('RETURN')
    

    ## replace brackets: ( --> [
    for i in range(len(ODEs[spec])):
        # print(i,ODEs[spec][i][0])
        if ODEs[spec][i][0] == 'F' or ODEs[spec][i][0] == 'D':
            # print('yes!')
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')

    ## replace final line
    last = ODEs[spec][-1]
    # print('last', last)
    while not last[0] == 'Y':
        ODEs[spec].remove(last)
        last = ODEs[spec][-1]
    # print('at the end',last)
    for i in range(5,len(last)):
        if last[i] == ')':
            stop = i
            break
    nb = last[5:stop]

    ODEs[spec][-1] = 'YDOT['+nb+']=F-(D*Y['+nb+'])'

    return ODEs

    

In [3]:
file = '/lhome/silkem/CHEM/src-IP-AP-HNR/code/acodes.f'


with open(file, 'r') as f:
    lines = f.readlines()

type(lines)

list

In [4]:
np.shape(lines)

(10519,)

In [5]:
spec_lines = lines[123:]

In [6]:
spec_lines = prep_lines(spec_lines)

In [7]:
spec_lines[583]

'*Y(17)*Y(17)*HNR+K(5224)*Y(17)*Y(88)*HNR+K(5225)*Y(17)*Y(88)*HNR+'

In [8]:
ODEs = dict()

for i, line in enumerate(spec_lines):
    if len(line) != 0 and line[0] == 'C':
        ODEs[line[2:]] = str('')
        print('i=',i, end = '\r')
        start = i + 1

        for j in range(start,start+1000):
            if j >= len(spec_lines):
                break
            if len(spec_lines[j]) != 0 and spec_lines[j][0] == 'C':
                # print(j, spec_lines[j])
                stop = j
                break
            
        for j in range(i+1,stop):
            ODEs[line[2:]] += spec_lines[j]

        i = stop +1




In [9]:

count = 0
for key in ODEs:
    if key == '':
        # del ODEs[key]
        count += 1
print('count=',count)

del ODEs['']

count= 1


In [10]:
for key in ODEs:
    if  key == '':
        print(key)

In [11]:
ODEs['C11+']

'      F=0.+K(1517)*Y(10)*Y(458)*HNR+K(1518)*Y(10)*Y(455)*HNR+K(2066)*Y(11)*Y(453)*HNR+K(2068)*Y(11)*Y(459)*HNR+K(5727)*Y(465)+K(6042)*Y(10)*Y(451)*HNR+K(6202)*Y(465)+K(6538)*Y(465)      D=0.+K(994)*X(1)*HNR+K(995)*X(1)*HNR+K(996)*X(1)*HNR+K(997)*X(1)*HNR      YDOT(466)=F-(D*Y(466))      RETURN      END'

In [12]:
ODEs = fortran_to_python(ODEs, 'C11+')

C11+


In [13]:
for key in ODEs:
    ODEs = fortran_to_python(ODEs, key)

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

In [15]:
ODEs['CO']

['F=0.+K[4]*Y[9]*Y[214]+K[4]*Y[9]*Y[214]+K[9]*Y[9]*Y[28]+K[57]*Y[11]*Y[26]+K[130]*Y[28]*Y[272]+K[131]*Y[28]*Y[385]+K[209]*Y[51]*Y[73]+K[242]*Y[54]*Y[73]+K[246]*Y[11]*Y[73]+K[261]*Y[17]*Y[73]+K[285]*Y[29]*Y[73]+K[289]*Y[14]*Y[73]+K[307]*Y[73]*Y[214]+K[308]*Y[73]*Y[89]+K[309]*Y[73]*Y[127]+K[310]*Y[73]*Y[77]+K[311]*Y[73]*Y[88]+K[312]*Y[73]*Y[104]+K[313]*Y[73]*Y[107]+K[478]*Y[37]*Y[73]+K[488]*Y[1]*Y[73]+K[497]*Y[60]*Y[73]+K[587]*Y[27]*Y[73]+K[602]*Y[35]*Y[73]+K[617]*Y[20]*Y[73]+K[682]*Y[28]*Y[73]+K[699]*Y[31]*Y[73]+K[764]*Y[154]+K[774]*Y[280]+K[841]*Y[178]+K[858]*Y[207]+K[864]*Y[320]+K[875]*Y[214]+K[886]*Y[89]',
 'F=F+K[906]*Y[77]+K[918]*Y[180]+K[953]*Y[331]+K[1038]*Y[156]*X[1]',
 'F=F+K[1050]*Y[299]*X[1]+K[1068]*Y[288]*X[1]+K[1172]*Y[175]*X[1]+K[1194]*Y[215]*X[1]+K[1201]*Y[192]*X[1]+K[1203]*Y[319]*X[1]+K[1233]*Y[201]*X[1]+K[1234]*Y[334]*X[1]+K[1249]*Y[91]*X[1]+K[1250]*Y[91]*X[1]+K[1261]*Y[212]*X[1]+K[1288]*Y[97]*X[1]+K[1309]*Y[164]*X[1]+K[1310]*Y[164]*X[1]+K[1349]*Y[79]*X[1]+K[1351]*Y[223